In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras
import autokeras as ak
from scikeras.wrappers import KerasClassifier
import numpy as np

In [38]:
# Load training dataset
df1 = pd.read_csv('./bioresponse/train.csv')
#df1['Activity'] = df1['Activity'].apply(np.int64)
#display(df1['Activity'].dtypes) 
#df1.describe() # Visualize

# Load testing dataset
kaggle_test = pd.read_csv('./bioresponse/test.csv')
# test.describe() # Visualize

# Separate labels from training dataset for fitting
labels = df1['Activity']
del df1['Activity']

# 80/20 train test split
x_train, x_test, y_train, y_test = train_test_split(df1, labels, test_size=0.2)
input_shape = x_train.shape
print("Shape of Input Data: " + str(input_shape))
print("Shape of Test Labels: " + str(y_test.shape))

#print(labels.describe())
#display(y_train.dtypes) 

Shape of Input Data: (3000, 1776)
Shape of Test Labels: (751,)


In [39]:
model = models.Sequential()
model.add(layers.Dense(200, input_dim=x_train.shape[1], activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(100, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
#model.add(layers.Softmax())
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 200)               355400    
                                                                 
 dense_25 (Dense)            (None, 100)               20100     
                                                                 
 dense_26 (Dense)            (None, 32)                3232      
                                                                 
 dense_27 (Dense)            (None, 1)                 33        
                                                                 
Total params: 378765 (1.44 MB)
Trainable params: 378765 (1.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)

#estimator = KerasClassifier(model=model, epochs=100, batch_size=32, verbose=1)


history = model.fit(
    x_train, y_train, epochs=200, validation_data=(x_test, y_test)
)


Epoch 1/200
94/94 [==============================] - 1s 11ms/step - loss: 0.4940 - accuracy: 0.7880 - val_loss: 0.6327 - val_accuracy: 0.7603
Epoch 2/200
94/94 [==============================] - 1s 8ms/step - loss: 0.5133 - accuracy: 0.7850 - val_loss: 0.6449 - val_accuracy: 0.7417
Epoch 3/200
94/94 [==============================] - 1s 8ms/step - loss: 0.4955 - accuracy: 0.7973 - val_loss: 0.6104 - val_accuracy: 0.7537
Epoch 4/200
94/94 [==============================] - 1s 8ms/step - loss: 0.5751 - accuracy: 0.7717 - val_loss: 1.3076 - val_accuracy: 0.6897
Epoch 5/200
94/94 [==============================] - 1s 7ms/step - loss: 0.5776 - accuracy: 0.7827 - val_loss: 0.9349 - val_accuracy: 0.6684
Epoch 6/200
94/94 [==============================] - 1s 7ms/step - loss: 0.6953 - accuracy: 0.7540 - val_loss: 0.6378 - val_accuracy: 0.7656
Epoch 7/200
94/94 [==============================] - 1s 7ms/step - loss: 0.7021 - accuracy: 0.7730 - val_loss: 0.9663 - val_accuracy: 0.6724
Epoch 8/200


In [41]:
prob_output = model.predict(kaggle_test)
print(prob_output)
print(prob_output.shape)
#arr = prob_output.reshape(1,2501)
df = pd.DataFrame(data=prob_output, columns=['PredictedProbability'])
df['MoleculeId'] = df.index + 1
df.head()
df.to_csv('submissionDL.csv',index=False)

79/79 [==============================] - 0s 2ms/step
[[0.9985868 ]
 [0.67792356]
 [0.27271444]
 ...
 [0.9334992 ]
 [0.86457944]
 [0.00753336]]
(2501, 1)
